![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Les perceptions sur la langue et l'appartenance<br><br>La mise en forme des données de sondage: Partie I

Bienvenue à ce notebook Jupyter. Ce notebook est gratuit et fait partie de la collection de Callysto, un projet promouvant la science des données dans les salles de classe de la 5e à la 12e année.

Dans ce notebook, nous utiliserons le python pour réaliser les différentes étapes d'analyse des données afin d'interpréter les données et de les représenter par les visualisations. Le python est un langage de programmation qui nous permet de communiquer avec l'ordinateur. Le python contient la langue anglaise, mais la grammaire diffère considérablement.

Pour démarrer le notebook et effectuer le code dans toutes les cellules, cliquez sur `Cell` dans le menu et sélectionnez `Run All`. Retournez en haut da la page après le chargement des cellules.

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

Veuillez envoyer un courriel à l'adresse contact@callysto.ca si vous rencontrez des problèmes avec le notebook.
<br><br>

## La démarche: les étapes d'analyses des données
1. **Questionner** - Formuler la question de recherche.
2. **Sélectionner** - Trouver les bibliothèques de python et les sources de données pour répondre à la question de recherche.
3. **Organiser** - Mettre en forme les données afin de les préparer pour l'exploration.
<br><br>

## Questionner

**Comment détermine-t'on l'impact des interactions sur les perceptions d'identité?**
<br><br>

## Sélectionner

### Le chargement des bibliothèques

Les bibliothèques sont les outils qui permettent à manipuler plus facilement le langage de programmation python.

Vous trouverez les notes sur le code dans la cellule en italique vert après le `#`.

In [ ]:
# Pour la manipulation de tableaux et les statistiques, importez la bibliothèque pandas sous l'alias pd
import pandas as pd

# Pour la manipulation de chiffres, importez la bibliothèque numpy sous l'alias np
import numpy as np

# Enlever les warnings
import warnings
warnings.filterwarnings("ignore")

### La source de données

**Données ouvertes** [Attribution (CC-BY 4.0)](https://www.donneesquebec.ca/licence/#cc-by)
<br><br>Ville de Montréal. (2020). Sondage Écho sur l'inclusion des personnes immigrantes. [Jeu de données]. Partenariat Données Québec. https://www.donneesquebec.ca/recherche/dataset/vmtl-sondage-inclusion-personnes-immigrantes

Pour en savoir plus sur le jeu de données, veuillez consulter la page web.

### Le chargement des données

Il y a deux fichiers disponibles : un pour les données et un pour les descriptions des variables.

Nous accédons aux données en format csv (comma-separated values) et les chargeons avec la bibliothèque pandas en utilisant la fonction `read_csv`. 

Nous chargeons l'ensemble de données dans la variable `data` et nous utilisons pandas sous l'alias `pd` pour lire à travers la fonction `read_csv`, dans le dossier `data`, le fichier `2020-sondage-echo-donnees.csv`.

#### Les données

In [ ]:
# Pour le chargement de données en format csv, un format délimité par des virgules
data = pd.read_csv("data/2020-sondage-echo-donnees.csv")
data

L'ensemble de données fait 2 503 lignes et 361 colonnes, ce qui fait environ 900 000 points de données, une estimation car `NaN` remplit plusieurs cellules. `NaN` veut dire *not a number* et indique que la cellule est vide.

Regardons aussi les descriptions des variables qui sont une forme de métadonnées, données sur les données !

#### Les métadonnées

In [ ]:
# Pour le chargement de metadonnées en format csv 
metadata = pd.read_csv("data/2020-sondage-echo-variables.csv")
metadata.head()

Remarquons que les métadonnées sont moins organisées dans le format tabulaire. Retirons les colonnes et lignes utiles liées à la question de recherche.

In [ ]:
questions = metadata[["champ", "description_question", "variable_reponse", "description_reponse"]]  # Garder les quatre premières colonnes
questions.columns = ["variable", "question", "reponse", "description"]  # Renommer les colonnes
questions.set_index("variable")  # Mettre le nom en index
questions = questions.loc[lambda df: (df["variable"].isin(["record", "sexe", "age", "NAISS", "LANGU", "LANGUr96oe", "MINO", "Q2r1", "Q2r2", "Q2r3", "Q2r4", "Q2r5", "Q2r6", "Q2r7", "Q2r8", "Q42", "Q46", "Q47", "Q47B", "Q52r1", "Q52r2", "Q52r3", "Q52r4", "Q52r5", "Q52r6", "Q52r7", "Q52r8", "Q52r9"]))]; # garder les lignes avec ces variables
questions.head() # Afficher les métadonnées visées

Nous listons les questions.

In [ ]:
questions.question.unique()

## Organiser

Nous récupérons l'information dans les colonnes qui s'appliquent à la question de recherche et nous affichons le tableau.

In [ ]:
selections = data[["record", "sexe", "age", "NAISS", "LANGU", "LANGUr96oe", "MINO", "Q2r1", "Q2r2", "Q2r3", "Q2r4", "Q2r5", "Q2r6", "Q2r7", "Q2r8", "Q42", "Q46", "Q47", "Q47B", "Q52r1", "Q52r2", "Q52r3", "Q52r4", "Q52r5", "Q52r6", "Q52r7", "Q52r8", "Q52r9"]]
selections.head()

Nous éliminons la sélection Je préfère de ne pas répondre dans les questions 42, 46, and 47.

In [ ]:
selections = selections.loc[lambda df: (df["Q42"] != 9) ];
selections = selections.loc[lambda df: (df["Q46"] != 9) ];
selections = selections.loc[lambda df: (df["Q47"] != 98) &(df["Q47"] != 99)];

Nous remplissons les NaNs dans la colonne `LANGUr96oe` avec l'information dans la colonne `LANGU`.

In [ ]:
selections["LANGUr96oe"] = selections["LANGUr96oe"].fillna(selections["LANGU"])
selections.head()

Nous supprimons la colonne `LANGU` qui a maintenant la même information que la colonne `LANGUr96oe`.

In [ ]:
selections_langue = selections.drop(columns=["LANGU"])
selections_langue.head()

Nous recodons les chiffres représentants la langue en mots et nous renommons la colonne `LANGUr96oe` en `LANGUE`.

In [ ]:
# Recodage de la langue
reco_langue = {
    1:"Français",
    2:"Anglais",
    96:"Autre",
    99:"Je préfère ne pas répondre",
}

selections_langue["LANGUr96oe"] = selections_langue["LANGUr96oe"].replace(reco_langue)
recodage = selections_langue.rename(columns={"LANGUr96oe": "LANGUE"})
recodage.head()

Nous recodons les autres variables.

In [ ]:
# Recodage du sexe
reco = {
    1:"Un homme",
    2:"Une femme",
    6:"Autre",
}

recodage["sexe"] = recodage["sexe"].replace(reco)

# Recodage de l'âge
reco = {
    0:"Moins de 18 ans",
    1:"De 18 à 24 ans",
    2:"De 25 à 34 ans",
    3:"De 35 à 44 ans",
    4:"De 45 à 54 ans",
    5:"De 55 à 64 ans",
    6:"De 65 à 74 ans",
    7:"75 ans ou plus",
}

recodage["age"] = recodage["age"].replace(reco)

# Recodage de la naissance
reco_NAISS = {
    1:"Oui",
    2:"Non",
}

recodage["NAISS"] = recodage["NAISS"].replace(reco_NAISS)

# Recodage de la minorité visible
reco_MINO = {
    1:"Oui",
    2:"Non",
    9:"Je préfère ne pas répondre",
}

recodage["MINO"] = recodage["MINO"].replace(reco_MINO)

# Inversion de l'échelle de la langue de la Q2r1 à la Q2r8
reco_Q2 = {
    1:5, "1":5,
    2:4, "2":4,
    4:2, "4":2,
    5:1, "5":1,

}

recodage.loc[:, "Q2r1":"Q2r8"] = recodage.loc[:, "Q2r1":"Q2r8"].replace(reco_Q2)

# Inversion de l'échelle de l'appartenance de la Q42
reco_Q42_str = {
    1:5, 
    2:4, 
    8:3, 
    3:2, 
    4:1, 
    "1":5,
    "2":4,
    "8":3,
    "3":2,
    "4":1,
}
    
recodage["Q42"] = recodage["Q42"].replace(reco_Q42_str)

#Recodage de l'échelle de l'appartenance de la Q46
reco_Q46 = {
    8:3, 
    3:4, 
    4:5, 
   
}

recodage["Q46"] = recodage["Q46"].replace(reco_Q46)

# Recodage de la Q47
reco_Q47 = {
    1:5,
    2:4,
    4:2,
    97:1,
}

recodage["Q47"] = recodage["Q47"].replace(reco_Q47)

# Recodage de la discrimination de la Q52r1 à la Q52r9
reco_Q52 = {
    1:"Oui",
    2:"Non",
    8:"Je ne sais pas",
    9:"Je préfère ne pas répondre",
}

recodage.loc[:, "Q52r1":"Q52r9"] = recodage.loc[:, "Q52r1":"Q52r9"].replace(reco_Q52)

recodage.head()

Nous décrivons les variables.

In [ ]:
recodage.describe(include=["object"])

Nous écrivons les données en un nouveau fichier.

In [ ]:
# Créer un nouveau fichier (2020-sondage-echo-recodage.csv) dans le dossier data
recodage.to_csv("data/2020-sondage-echo-recodage.csv") 

## Les variables visées après le recodage

|Variable     |Question |Réponse  |
|:-------------|:-------------|:-------------|
|record|identifiant unique du répondant||
|sexe|Vous identifiez-vous comme:|Un homme, Une femme|6 Autre|
|age|Quel âge avez-vous?|Moins de 18 ans, De 18 à 24 ans, De 25 à 34 ans, De 35 à 44 ans, De 45 à 54 ans, De 55 à 64 ans, De 65 à 74 ans, 75 ans ou plus|
|NAISS|NAISSANCE: Êtes-vous né au Canada?|Oui,  Non|
|LANGUE|LANGUE: Quelle est la langue que vous avez apprise en premier lieu à la maison dans votre enfance et que vous comprenez toujours?|Français, Anglais, Autre, Je préfère ne pas répondre|
|MINO|Considérez-vous appartenir à une minorité visible?|Oui, Non, Je préfère ne pas répondre|
|Q2r1|Je comprends le français|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q2r2|Je parle le français|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q2r3|Je lis en français|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q2r4|J’écris en français|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q2r5|Je comprends l’anglais|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q2r6|Je parle l’anglais|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q2r7|Je lis l’anglais|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q2r8|J’écris en anglais|5 Parfaitement, 4 Très bien, 3 Plutôt bien, 2 Pas très bien, 1 Pas du tout|
|Q42|Vous sentez-vous Montréalais.e?|5 Oui, je me sens tout à fait Montréalais.e, 4 Oui, je me sens plutôt Montréalais.e, 3 Je ne sais pas, 2 Je ne me sens pas vraiment, 1 Je ne me sens pas du tout Montréalais.eMontréalais.e|
|Q46|Vous sentez-vous isolé à Montréal?|1 Beaucoup, 2 Assez, 3 Je ne sais pas, 4 Peu, 5 Pas du tout|
|Q47|Parmi tous vos amis, combien d’entre eux/elles ont la même langue maternelle que vous?|Tous/Toutes, La majorité, À peu près la moitié, Quelques-uns / Quelques-unes, Aucun(e), Je ne sais pas, Je préfère ne pas répondre|
|Q52r1|Votre sexe? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r2|Votre appartenance ethnique ou nationale? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r3|La couleur de votre peau? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r4|Votre condition sociale? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r5|Votre religion? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r6|Votre orientation sexuelle? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r7|Votre âge? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r8|Une limitation fonctionnelle? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|
|Q52r9|Votre langue ou votre accent? - Au cours des cinq dernières années, ou depuis votre arrivée à Montréal, croyez-vous avoir été victime de discrimination ou traité injustement par d’autres personnes, à Montréal, à cause de...|Oui, Non, Je ne sais pas, Je préfère ne pas répondre|

# La fin

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)